# Latent Dirichlet Allocation

In [1]:
import pandas as pd

data = pd.read_csv('data', sep=",", header=None)

data.columns = ['text']

data.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


The data is a collection of emails that are not labelled. Let's try extract topics from them!

## Preprocessing 

👇 You're used to it by now... Clean up! Store the cleaned text in a new dataframe column "clean_text".

In [2]:
from nltk.corpus import stopwords 
import string
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize 

def clean (text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '') # Remove Punctuation
    text = text.lower() # Lower Case
    tokenized = word_tokenize(text) # Tokenize
    words_only = [word for word in tokenized if word.isalpha()] # Remove numbers
    stop_words = set(stopwords.words('english')) # Make stopword list
    without_stopwords = [word for word in words_only if not word in stop_words] # Remove Stop Words
    lemma=WordNetLemmatizer() # Initiate Lemmatizer
    lemmatized = [lemma.lemmatize(word) for word in without_stopwords] # Lemmatize
    new_text = ' '.join(lemmatized)
    return new_text

# Apply to all texts
data['clean_text'] = data.text.apply(clean)
# data['clean_text_str'] = data['clean_text'].astype('str')

data.head()

,text,clean_text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...,gldcunixbcccolumbiaedu gary l dare subject sta...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...,atterlepvelaacsoaklandedu cardinal ximenez sub...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...,minerkuhubccukansedu subject ancient book orga...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...,atterlepvelaacsoaklandedu cardinal ximenez sub...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...,vzhivovsuperiorcarletonca vladimir zhivov subj...


## Latent Dirichlet Allocation model

👇 Train an LDA model to extract potential topics.

In [3]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer().fit(data['clean_text'])

data_vectorized = vectorizer.transform(data['clean_text'])

lda_model = LatentDirichletAllocation(n_components=2).fit(data_vectorized)

## Visualize potential topics

👇 The function to print the words associated with the potential topics is already made for you. You just have to pass the correct arguments!

In [4]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])
        print('-'*100)
        

print_topics(lda_model, vectorizer)

Topic 0:
[('god', 35.79086452880034), ('game', 27.074517396183545), ('would', 26.170523302599392), ('team', 25.784280281659242), ('one', 24.348146079915697), ('line', 23.26754485955383), ('subject', 23.067321087086565), ('christian', 22.4653378626338), ('organization', 22.16626630667141), ('university', 22.059226378610187)]
----------------------------------------------------------------------------------------------------
Topic 1:
[('cell', 1.795795572520442), ('sturm', 0.9919793685390939), ('singapore', 0.960612516661315), ('dee', 0.903739719121338), ('zazula', 0.7495506441370552), ('returnpath', 0.7375711469771868), ('xmailer', 0.7375711468124797), ('dohertylukacgladcs', 0.7375711467329908), ('dohertyl', 0.7375711466405994), ('dohertyldcsglaacuk', 0.7375711461329668)]
----------------------------------------------------------------------------------------------------


## Predict topic of new text

👇 You can now use your LDA model to predict the topic of a new text. First, use your vectorizer to vectorize the example. Then, use your LDA model to predict the topic of the vectorized example.

In [5]:
example = ["My team performed poorly last season. Their best player was out injured and only played one game"]

example_vectorized = vectorizer.transform(example)

lda_vectors = lda_model.transform(example_vectorized)

print("topic 0 :", lda_vectors[0][0])
print("topic 1 :", lda_vectors[0][1])

topic 0 : 0.863367451224021
topic 1 : 0.13663254877597902
